In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

# Function to scrape a single recipe with error handling
def scrape_recipe(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        recipe_content = response.text
        recipe_soup = BeautifulSoup(recipe_content, 'html.parser')
        ingredients_section = recipe_soup.find('section', class_='o-Ingredients')
        if ingredients_section:
            ingredients = ingredients_section.find_all('span', class_='o-Ingredients__a-Ingredient--CheckboxLabel')
            ingredient_list = [ingredient.get_text() for ingredient in ingredients]
        else:
            ingredient_list = []
        return ingredient_list
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return []

# Function to scrape recipes from a given main page URL with pagination and error handling
def scrape_recipes_from_page(base_page_url, start_page=0):
    page_number = start_page
    all_recipe_data = []
    base_url = 'https:'
    letter = base_page_url.split('/')[-1]
    checkpoint_file = f'recipes_checkpoint_{letter}.csv'

    while True:
        page_url = f"{base_page_url}/p/{page_number}" if page_number > 0 else base_page_url
        try:
            response = requests.get(page_url)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {page_url}: {e}")
            break
        
        web_content = response.text
        soup = BeautifulSoup(web_content, 'html.parser')

        # Find all recipe list items
        recipe_items = soup.find_all('li', class_='m-PromoList__a-ListItem')

        if not recipe_items:
            break

        # Extract the URLs and names of the recipes
        recipes = []
        for item in recipe_items:
            anchor = item.find('a')
            recipe_name = anchor.get_text()
            recipe_url = base_url + anchor['href']
            recipes.append({'name': recipe_name, 'url': recipe_url})

        # Scrape each recipe and store in a DataFrame
        for recipe in recipes:
            print(f"Scraping recipe: {recipe['name']} from {page_url}")
            ingredients = scrape_recipe(recipe['url'])
            all_recipe_data.append({'name': recipe['name'], 'url': recipe['url'], 'ingredients': ingredients})
            time.sleep(.5)  # Add a delay between requests

        # Save checkpoint
        df_checkpoint = pd.DataFrame(all_recipe_data)
        df_checkpoint.to_csv(checkpoint_file, index=False)
        
        page_number += 1
        time.sleep(2)  # Add a delay between page requests
    
    return all_recipe_data

# Function to scrape all pages for a specific letter and handle checkpoints
def scrape_all_pages_for_letter(letter):
    page_url = f'https://www.foodnetwork.com/recipes/recipes-a-z/{letter}'
    checkpoint_file = f'recipes_checkpoint_{letter}.csv'
    if os.path.exists(checkpoint_file):
        df_checkpoint = pd.read_csv(checkpoint_file)
        all_recipes_data = df_checkpoint.to_dict('records')
        last_page = 0
        for recipe in all_recipes_data:
            try:
                last_page = max(last_page, int(recipe['url'].split('/p/')[1].split('/')[0]))
            except (IndexError, ValueError):
                pass
    else:
        all_recipes_data = []
        last_page = 0

    try:
        print(f"Scraping page: {page_url}")
        all_recipes_data += scrape_recipes_from_page(page_url, start_page=last_page)
        time.sleep(5)  # Add a delay between page requests
    except Exception as e:
        print(f"An error occurred: {e}")

    # Convert to a DataFrame and save
    df = pd.DataFrame(all_recipes_data)
    df_expanded = df['ingredients'].apply(pd.Series)
    df_expanded = pd.concat([df['name'], df['url'], df_expanded], axis=1)
    df_expanded.to_csv(f'recipes_{letter}_full.csv', index=False)
    print(f"DataFrame saved as recipes_{letter}_full.csv")

# Get user input for the letter
letter = input("Enter the letter for which you want to scrape recipes: ").lower()

# Start scraping
scrape_all_pages_for_letter(letter)


Enter the letter for which you want to scrape recipes:  a


Scraping page: https://www.foodnetwork.com/recipes/recipes-a-z/a
Scraping recipe: "A Bologna" - Calamari, Scallops and Clams with Roasted Fingerlings and Arugula Salad from https://www.foodnetwork.com/recipes/recipes-a-z/a
Scraping recipe: "A Piece of Turkey" Easy Roasted Thanksgiving Turkey from https://www.foodnetwork.com/recipes/recipes-a-z/a
Scraping recipe: "A Star is Born" Cupcakes from https://www.foodnetwork.com/recipes/recipes-a-z/a
Scraping recipe: "Aarti Party" aka Savory Sling from https://www.foodnetwork.com/recipes/recipes-a-z/a
Scraping recipe: "Acqua Pazza" with Sea Bass and Clams from https://www.foodnetwork.com/recipes/recipes-a-z/a
Scraping recipe: "All Crabmeat" Crab Cakes from https://www.foodnetwork.com/recipes/recipes-a-z/a
Scraping recipe: "Almost Famous" Corn Pudding from https://www.foodnetwork.com/recipes/recipes-a-z/a
Scraping recipe: "Almost Famous" Strawberry Lemonade from https://www.foodnetwork.com/recipes/recipes-a-z/a
Scraping recipe: "Ayuh" Seafood Pa

Scraping recipe: Sagaponack Corn Pudding from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Sage Advice from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Sage Butter Macaroni and Four Cheese from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Sage Creamed Corn Bruschetta with Prosciutto from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Sage Dressing from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Sage Fried Chicken and Bacon Waffles from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Sage Limeade from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Sage Pesto Crusted Turkey Tenderloin from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Sage Potatoes Au Gratin from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Sage Potatoes Au Gratin from https://www.foodnetwork.com

Scraping recipe: Salad of Roasted Beets and Arugula with Blue Cheese Dressing from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Salad of Roasted Beets and Arugula with Blue Cheese Dressing from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Salad of Shaved Artichokes and Sunchokes with Grapefruit and Chicories from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Salad of Spring Mesclun, Blossoms, and Wild Greens, with Lovage-Allium Dressing from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Salad of Spring Mesclun, Blossoms, and Wild Greens, with Lovage-Allium Dressing from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Salad of The Greens with Fennel, Mint and Orange from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Salad of Winter Greens from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/2
Scraping recipe: Salad of the Goddesses wit

Scraping recipe: Salmon Beurre Legre from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/3
Scraping recipe: Salmon Bites with Warm Brussels Sprout Slaw from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/3
Scraping recipe: Salmon Burger from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/3
Scraping recipe: Salmon Burger from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/3
Scraping recipe: Salmon Burgers from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/3
Scraping recipe: Salmon Burgers from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/3
Scraping recipe: Salmon Burgers with Caesar Slaw from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/3
Scraping recipe: Salmon Burgers with Curry Mayo and Asian Slaw from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/3
Scraping recipe: Salmon Burgers with Green Apple Slaw from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/3
Scraping recipe: Salmon Burgers with Lemon-Caper Aioli and Kale Slaw from https

...

Scraping recipe: Seafood Grilled in Foil from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seafood Gumbo from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seafood Gumbo from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seafood Gumbo from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seafood Gumbo from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seafood Gumbo from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seafood Gumbo from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seafood Gumbo from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seafood Hot Dogs with Sweet Mustard Sauerkraut from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seafood Jambalaya from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seafood Jambalaya from htt

Scraping recipe: Seared Ahi Tuna with Asian Slaw from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seared Ahi Tuna with Ginger Beets and Seaweed Salad from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seared Ahi with Passion Fruit-Shrimp Salsa from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seared Arctic Char with Kumquat Compote from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seared Atlantic Salmon Fillets from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seared Beef Tornadoes from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seared Beef Tournedos with Herb-Roasted Potatoes and Sauce au Poivre (Delmonico Steakhouse) from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping recipe: Seared Black Bass with Bitter Greens, Grapefruit and Feta Salad from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/14
Scraping

Scraping recipe: Seared Lamb Chops with Roasted Radishes and Haricots Verts, Black Olives, Pine Nuts, Fennel and Scalloped Potato Cake with Goat Cheese and Dill from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Lima Beans with Roasted Pumpkin Seeds from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Long Island Duck Breast and Foie Gras Sauce Red Cabbage Slaw and Warm Yukon Gold Potato and Onion Salad Chinese and Balsamic Vinegar Emulsion from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Macadamia Crusted Tuna from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Madras Haddock with Broccoli Sauce from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Mahi Mahi with Grilled Mango-Pineapple Salsa, Green Rice, and Black Beans from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Mahi-Mahi with Mango Sa

Scraping recipe: Seared Salmon, Carrot Tortellini, Red Pepper-Cumin Sauce from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Salmon, Carrot Tortellini, Red Pepper-Cumin Sauce from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Salmon, Carrot Tortellini, Red Pepper-Cumin Sauce from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Salmon, Carrot Tortellini, Red Pepper-Cumin Sauce from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Salmon, Carrot Tortellini, Red Pepper-Cumin Sauce from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Salmon, Carrot Tortellini, Red Pepper-Cumin Sauce from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Salmon, Carrot Tortellini, Red Pepper-Cumin Sauce from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Salmon, Carrot Tortellini, Red Pepper-

Scraping recipe: Seared Spicy Tuna on Basil Parmesan Crisps With Apricot Jam from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Steak With Chard Salad from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Steak and Green Beans with Herbed Butter from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Steaks with Bourbon-Thyme Pan Sauce, Rice Pilaf with Cranberries and Almonds from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Strip Steak with Jalapeño Cream Sauce from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Strip Steaks with North African-Spiced Butter from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/15
Scraping recipe: Seared Striped Bass on Chive Whipped Potatoes with Truffle Vinaigrette from https://www.foodnetwork.com/recipes/recipes-a-z/s/p/16
Scraping recipe: Seared Tilapia with Asparagus and Spicy Mint Gremolat